In [71]:
import requests, json
import re
from datetime import datetime
import numpy

In [95]:
apistarturl = 'https://api.weather.gov'

def get_location_url_from_coords(lat,lon):
    return apistarturl + '/points/' + str(lat) + ',' + str(lon)


# HERE IS WHERE YOU CHANGE THE LAT / LON, FOR THE TEST WE'RE GOING TO USE A FILE WITH A LIST OF CITIES

r = requests.get('https://gist.githubusercontent.com/Miserlou/c5cd8364bf9b2420bb29/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json')
g = json.loads(r.text)

# WE'RE GOING TO TEST 10 RANDOM CITIES
def selectRandomCities(city):
    return numpy.random.choice(city, 10)

k = selectRandomCities(g)
j = k.tolist()
j


[{'city': 'Romeoville',
  'growth_from_2000_to_2013': '79.5%',
  'latitude': 41.6475306,
  'longitude': -88.0895061,
  'population': '39650',
  'rank': '931',
  'state': 'Illinois'},
 {'city': 'Richardson',
  'growth_from_2000_to_2013': '13.2%',
  'latitude': 32.9483335,
  'longitude': -96.7298519,
  'population': '104475',
  'rank': '275',
  'state': 'Texas'},
 {'city': 'Cincinnati',
  'growth_from_2000_to_2013': '-10.1%',
  'latitude': 39.1031182,
  'longitude': -84.5120196,
  'population': '297517',
  'rank': '65',
  'state': 'Ohio'},
 {'city': 'Surprise',
  'growth_from_2000_to_2013': '281.9%',
  'latitude': 33.6292337,
  'longitude': -112.3679279,
  'population': '123546',
  'rank': '216',
  'state': 'Arizona'},
 {'city': 'Concord',
  'growth_from_2000_to_2013': '47.4%',
  'latitude': 35.4087517,
  'longitude': -80.579511,
  'population': '83506',
  'rank': '383',
  'state': 'North Carolina'},
 {'city': 'Enid',
  'growth_from_2000_to_2013': '8.1%',
  'latitude': 36.3955891,
  'lon

In [96]:
def get_api_products(cities):
    
    for i in j:
        url = get_location_url_from_coords(j[0]['latitude'],j[0]['longitude'])
        r = requests.get(url)
    location_json_obj = json.loads(r.text)
    
    
    office = location_json_obj['properties']['gridId']
    station = 'K'+ office
    products_in_station_url = apistarturl + '/products/locations/' + office + '/types'
    r = requests.get(products_in_station_url)
    products_at_station = json.loads(r.text)['@graph']
    products_at_station
    
    #THIS IS AN IMPORTANT LINE
    #v v v v vv v v v v v v v  
    desired_product_code = 'PFM'
    # ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^
    #HERE IS WHERE YOU CHANGE THE PRODUCT CODE TO BE THE THREE LETTER PRODUCT OF INTEREST

    for product in products_at_station:
        if product['productCode'] == desired_product_code:
            print(product)
    recentProductURL = apistarturl + '/products/types/' + desired_product_code
    r = requests.get(recentProductURL)
    recentProductsJSON = json.loads(r.text)

    station_products = []
    for product in recentProductsJSON['@graph']:
        if product['issuingOffice'] == station:
            station_products.append(product)
    station_products[0]
    
    #HERE IS WHERE THE PRODUCTS AVAILABLE ARE REDUCED TO THE MOST RECENT ONE

    t1 = datetime.strptime(station_products[0]['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
    t2 = datetime.strptime(station_products[1]['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
    most_recent_date = datetime(1,1,1)
    most_recent_product = ''
    for product in station_products:
        product_date = datetime.strptime(product['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
        if most_recent_date < product_date:
            most_recent_product = product
            most_recent_date = product_date
            
    most_recent_product_url = most_recent_product['@id']
    r =requests.get(most_recent_product_url)
    product_json = json.loads(r.text)
    
    return product_json['productText']


In [105]:
def returnthree_hr_timeseries_json_from_pfm(PFM):
    
    three_hr_timeseries_json = []
    location_coordinates = re.findall(r'.....N.......W E',PFM)
    location_texts = re.split(r'.....N.......W E', PFM)
    
    for index in range(len(location_coordinates)):
        
        time_series_dict = {}
        time_series_dict['location'] = location_coordinates[index]
        
        #date parser
        datesh = re.findall(r'Date         ................................................................', location_texts[1+index])
        date3h = re.findall(r'Date         ...................',datesh[0])
        start_d = date3h[0].replace('Date         ','')
        start_date = start_d.replace('  ','')
        time_series_dict['start_date'] = start_date

        #hour parser
        hours3h = re.findall(r'T 3hrly     .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..',location_texts[1+index])
        hour_csv = hours3h[0].replace('T 3hrly     ','')
        hours = hour_csv.split(' ')
        
        #go through hours and make them additional
        hours_added = []
        daycounter = 0
        lasthour = int(hours[0])
        for hour in hours:
            if int(hour) < lasthour:
                daycounter += 1
            hours_added.append(int(hour)+ daycounter * 24)
            lasthour = int(hour)
        
        #temp parser:
        
        temp = re.findall(r'Temp         ..................................................................', location_texts[1+index])
        if len(temp) > 0:
            forec = temp[0].replace('Temp         ','')
            temps = [forec[i:i+3] for i in range(0, len(forec), 3)]
        else:
            temp = re.findall(r'TEMP         ..................................................................', location_texts[1+index])
            forec = temp[0].replace('TEMP         ','')
            temps = [forec[i:i+3] for i in range(0, len(forec), 3)]
            
        #dew point parser:
        
        dew = re.findall(r'Dewpt         .................................................................', location_texts[1+index])
        if len(dew) > 0:
            dewp = dew[0].replace('Dewpt         ','')
            dewpoint = [dewp[i:i+3] for i in range(0, len(dewp), 3)]
        else:
            dew = re.findall(r'DEWPT         .................................................................', location_texts[1+index])
            dewp = dew[0].replace('DEWPT         ','')
            dewpoint = [dewp[i:i+3] for i in range(0, len(dewp), 3)]
            
        #wind direction parser:
        
        wind = re.findall(r'Wind dir      .................................................................', location_texts[1+index])
        if len(wind) > 0:
            windDi = wind[0].replace('Wind dir      ','')
            windDirection= [windDi[i:i+3] for i in range(0, len(windDi), 3)]
        else:
            wind = re.findall(r'WIND DIR      .................................................................', location_texts[1+index])
            windDi = wind[0].replace('WIND DIR      ','')
            windDirection= [windDi[i:i+3] for i in range(0, len(windDi), 3)]
            
        #windspeed parser:
        
        winds = re.findall(r'Wind spd      .................................................................',location_texts[1+index])
        if len(winds) > 0:
            windsp = winds[0].replace('Wind spd      ','')
            windSpeed = [windsp[i:i+3] for i in range(0, len(windsp), 3)]
        else:
            winds = re.findall(r'WIND SPD      .................................................................',location_texts[1+index])
            windsp = winds[0].replace('WIND SPD      ','')
            windSpeed = [windsp[i:i+3] for i in range(0, len(windsp), 3)]

        #clouds parser
        
        cl= re.findall(r'Clouds        .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..', location_texts[1+index])
        if len(cl) > 0:
            clo = cl[0].replace('Clouds        ','')
            clouds = [clo[i:i+3] for i in range(0, len(clo), 3)]
        else:
            cl= re.findall(r'CLOUDS        .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..', location_texts[1+index])
            clo = cl[0].replace('CLOUDS        ','')
            clouds = [clo[i:i+3] for i in range(0, len(clo), 3)]            
        
        #Relative humidity parser
        rh1 = re.findall(r'RH            .................................................................', location_texts[1+index])
        rh2 = rh1[0].replace('RH            ','')
        rh = [rh2[i:i+3] for i in range(0, len(rh2), 3)]
    
        
        #Probability of Precipitation: The next three items are not part of the 3 hour time series
        prec = re.findall(r'PoP 12hr                    ...................................................', location_texts[1+index])
        if len(prec)> 0:
            precip = prec[0].replace('PoP 12hr                     ','')
            precipit = precip.replace('          ',',')
            pop = precipit.split(',')
            time_series_dict['pop12H'] = pop
        else:
            prec = re.findall(r'POP 12HR                    ...................................................', location_texts[1+index])
            precip = prec[0].replace('POP 12HR                     ','')
            precipit = precip.replace('          ',',')
            pop = precipit.split(',')
            time_series_dict['pop12H'] = pop

            
        #Quantitative Precipitation Forecast
        quantit= re.findall(r'QPF 12hr                 ......      ......      ......      ......      ......',location_texts[1+index])
        if len(quantit) > 0:
            quantiprec = quantit[0].replace('QPF 12hr                 ','')
            quantiprecip = quantiprec.replace('      ',',')
            qpf = quantiprecip.split(',')
            time_series_dict['QPF12H'] = qpf
        else: 
            quantit= re.findall(r'QPF 12HR                 ......      ......      ......      ......      ......',location_texts[1+index])
            quantiprec = quantit[0].replace('QPF 12HR                 ','')
            quantiprecip = quantiprec.replace('      ',',')
            qpf = quantiprecip.split(',')
            time_series_dict['QPF12H'] = qpf
            
        #Not all of the stations measure the probability of snow e.g. Phoenix, Houston. If your PFM doesn't contain this part of
        #the include an if to check the snow
 
        s = re.findall(r'Snow 12hr                .......     .......     .......                       ',location_texts[1+index])
        if len(s) > 0:
            snow = s[0].replace('Snow 12hr                 ','')
            snow1 = snow.replace('                        ','')
            snow12 = snow1.replace('       ',',')
            snow12h = snow12.split(',')
            time_series_dict['Snow12H'] = snow12h
        
        s = re.findall(r'SNOW 12HR                .......     .......     .......                       ',location_texts[1+index])
        if len(s) > 0:
            snow = s[0].replace('SNOW 12HR                 ','')
            snow1 = snow.replace('                        ','')
            snow12 = snow1.replace('       ',',')
            snow12h = snow12.split(',')
            time_series_dict['Snow12H'] = snow12h

        #align hour and temps
        
        ts = []
        
        for index2 in range(len(temps)):
    
            ts.append([hours_added[index2],temps[index2],dewpoint[index2], rh[index2], windDirection[index2], windSpeed[index2], clouds[index2]])

        time_series_dict['timeseries'] = ts
        three_hr_timeseries_json.append(time_series_dict)
        
    return three_hr_timeseries_json

In [103]:
citylist = []
for index in range(0,len(j)):
    PFM = get_api_products(j)
    product_json = returnthree_hr_timeseries_json_from_pfm(PFM)
    
    citylist.append(product_json[index])


{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}


In [106]:
citylist

[{'location': '41.99N  87.91W E',
  'start_date': '07/20/22Thu',
  'pop12hours': [' 0', '20', '10', ' 5', '50'],
  'QPF12h': ['     0', '  0.01', '  0.01', '     0', '  0.07'],
  'Snow12H': ['00-00', '00-00', '00-00'],
  'timeseries': [[15, '   ', '   ', '   ', '   ', '   ', '   '],
   [18, ' 87', '64 ', '46 ', ' W ', '18 ', 'SC '],
   [21, ' 81', '64 ', '56 ', ' W ', '12 ', 'FW '],
   [24, ' 77', '63 ', '62 ', ' W ', '10 ', 'CL '],
   [27, ' 72', '62 ', '71 ', ' W ', ' 9 ', 'CL '],
   [30, ' 70', '61 ', '73 ', ' W ', ' 9 ', 'CL '],
   [33, ' 81', '61 ', '51 ', ' W ', '10 ', 'CL '],
   [36, ' 89', '61 ', '39 ', ' W ', '13 ', 'FW '],
   [39, ' 93', '62 ', '36 ', ' W ', '17 ', 'SC '],
   [42, ' 91', '64 ', '41 ', ' W ', '13 ', 'SC '],
   [45, ' 84', '63 ', '49 ', ' W ', ' 9 ', 'FW '],
   [48, ' 77', '62 ', '60 ', ' W ', ' 5 ', 'FW '],
   [51, ' 72', '61 ', '68 ', ' W ', ' 5 ', 'FW '],
   [54, ' 69', '60 ', '73 ', ' W ', ' 5 ', 'FW '],
   [57, ' 79', '59 ', '50 ', ' W ', ' 5 ', 'SC '],
  